In [9]:
## 인프라_도로 시군구 카운트

def make_infra_file(facility_file_name, output_file_name):
    #directory path
    local_dir_path = '.\\data\\infra\\local'
    fac_dir_path = '.\\data\\infra\\facility'
    #file path
    local_file_name = '' + '.'
    facility_file_name = facility_file_name + '.xlsx'

    #output directory
    out_put_dir_path = '.\\infra'

    # import local_codes # code data
    import _codes
    import pandas as pd
    import math
    import _util

    df = pd.read_excel(fac_dir_path + '\\' + facility_file_name)
    # print(df.columns)

    #dataframe화 하기
    new_list = []
    for idx, row in df.iterrows():
        #맨 마지막줄 멈춤
        if idx == len(df) - 1: break
        #앞에 쓸모 없는 줄 넘김
        if idx <= 5: continue

        sido_nm = row['Unnamed: 8']
        sido_cd = _util.sido_cd_map.get(sido_nm, None)

        cpt_dt = row['Unnamed: 11']
        if not math.isnan(cpt_dt): 
            cpt_dt = str(cpt_dt)
            year = int(cpt_dt[0:4])
        else: year = 2000

        sido_sgg_cd_map = {}
        # if year <= 2000: 
        #     sido_sgg_cd_map = local_codes.process_sgg_codes(local_codes.sgg_codes_2000, 2)
        #     year = 2000
        # elif year <= 2005:
        #     sido_sgg_cd_map = local_codes.process_sgg_codes(local_codes.sgg_codes_2005, 2)
        #     year = 2005
        # elif year <= 2010: 
        #     sido_sgg_cd_map = local_codes.process_sgg_codes(local_codes.sgg_codes_2010, 2)
        #     year = 2010
        # elif year <= 2015: 
        #     sido_sgg_cd_map = local_codes.process_sgg_codes(local_codes.sgg_codes_2015, 2)
        #     year = 2015
        # elif year <= 2020: 
        #     sido_sgg_cd_map = local_codes.process_sgg_codes(local_codes.sgg_codes_2020, 2)
        #     year = 2020
        if year <= 2023: 
            sido_sgg_cd_map = _util.process_sgg_codes(_codes.sgg_codes_2023, 2)
            # year = 2023

        sgg_nm = row['Unnamed: 9']
        sido = sido_sgg_cd_map.get(sido_cd, {})
        sido = dict(sido)
        sgg_cd = sido.get(sgg_nm, None)
        # if sgg_nm == '전라북도': sgg_cd = '35'

        new_list.append({
            'fac_nm': row['Unnamed: 1']
            , 'fac_sort': row['Unnamed: 2']
            , 'admin_sys1': row['Unnamed: 3']
            , 'admin_sys2': row['Unnamed: 4']
            , 'sido_cd': f'`{sido_cd}'
            , 'sido_nm': sido_nm
            , 'sgg_cd': f'`{sgg_cd}'
            , 'sgg_nm': sgg_nm
            , 'emd_nm': row['Unnamed: 10']
            , 'cpt_dt': f'`{cpt_dt}'
            , 'year': f'`{year}'
        })

    col_list = []
    for col in df.columns:
        if col.count('fac_sort') > 0: continue
        col_list.append(col)

    df = pd.DataFrame(new_list)
    df.to_excel(out_put_dir_path + '\\' + output_file_name + '.xlsx', index=False)

list_file_nm = ['도로', '철도', '항만, 어항', '공항', '전기', '가스', '열공급', '통신', '공동구', '송유', '하천', '저수지', '댐', '하수도']
# print(len(list_file_nm))
for idx in range(len(list_file_nm)):
    file_nm = list_file_nm[idx]
    if file_nm == '전기': continue
    make_infra_file(f'{idx + 1}. 인프라 총조사 결과_시설별 현황({file_nm})', file_nm)

In [2]:
import _util
def create_table(columns):
    columns = list(columns)
    for idx in range(len(columns)):
        columns[idx] = f'"{columns[idx]}"'

        if str(columns[idx]).count('fac_nm') > 0: columns[idx] = columns[idx] + ' varchar(100)'
        if str(columns[idx]).count('fac_sort') > 0: columns[idx] = columns[idx] + ' varchar(10)'
        if str(columns[idx]).count('admin_sys1') > 0: columns[idx] = columns[idx] + ' varchar(20)'
        if str(columns[idx]).count('admin_sys2') > 0: columns[idx] = columns[idx] + ' varchar(10)'
        if str(columns[idx]).count('sido_nm') > 0: columns[idx] = columns[idx] + ' varchar(10)'
        if str(columns[idx]).count('sido_cd') > 0: columns[idx] = columns[idx] + ' varchar(2)'
        if str(columns[idx]).count('sgg_nm') > 0: columns[idx] = columns[idx] + ' varchar(10)'
        if str(columns[idx]).count('sgg_cd') > 0: columns[idx] = columns[idx] + ' varchar(5)'
        if str(columns[idx]).count('emd_nm') > 0: columns[idx] = columns[idx] + ' varchar(10)'
        if str(columns[idx]).count('cpt_dt') > 0: columns[idx] = columns[idx] + ' varchar(8)'
        if str(columns[idx]).count('year') > 0: columns[idx] = columns[idx] + ' varchar(4)'

    sql = f'''create table if not exists infra(
        {', '.join(columns)}
    )
    '''

    _util.execute_sql(sql)

#insert
def insert_data(df, columns):
    #column setting
    columns = list(columns)
    for idx in range(len(columns)):
        columns[idx] = f'"{columns[idx]}"'
    
    #to use dataframe for clause
    for idx, row in df.iterrows():
        if row['sido_cd'] == '`None': continue
        row = row.tolist()
            
        for idx in range(len(columns)):
            # elif idx == len(columns) - 1: row[idx] = f'st_geomfromtext(\'{row[idx]}\', 4326)' #wkt string data => geometrys
            if str(row[idx]).count('`') > 0: row[idx] = str(row[idx]).replace('`', '')
            
            row[idx] = f'\'{str(row[idx])}\''
            if row[idx] == 'None': row[idx] = '\'\''

        sql = f'''
            INSERT INTO infra(
                {','.join(columns)})
            VALUES(
                {', '.join(row)}
            )
        '''
        
        _util.execute_sql(sql)

In [12]:
import pandas as pd

out_put_dir_path = '.\\infra'

list_file_nm = ['도로', '철도', '항만, 어항', '공항', '전기', '가스', '열공급', '통신', '공동구', '송유', '하천', '저수지', '댐', '하수도']
for file_nm in list_file_nm:
    if file_nm == '전기': continue
    df = pd.read_excel(out_put_dir_path + '\\' + file_nm + '.xlsx')
    create_table(df.columns)
    insert_data(df, df.columns)

In [3]:
import _util

years = [2000, 2005, 2010, 2015, 2020]

def drop_view(year, key):
    sql = f'''
    DO $$
    BEGIN
        IF EXISTS (SELECT 1 FROM pg_matviews WHERE matviewname = 'v_{key}_infra{year}_5179') THEN
        drop materialized view v_{key}_infra{year}_5179;
        END IF;
    END $$;
    '''
    _util.execute_sql(sql)

infras = {
    'road': 'admin_sys1 like \'%도로%\'',
    'cmm_tunnel': 'admin_sys1 like \'%공동구%\'',
    'gas': 'fac_sort like \'%가스%\'',
    'rail': 'admin_sys1 like \'%철도%\'',
    'cd': 'admin_sys1 like \'%통신구%\'',
    'heat': 'fac_sort like \'%열수송관%\''
}

def infra_sql(year, key):
    sql = f'''
    DO $$
    BEGIN
        IF NOT EXISTS (SELECT 1 FROM pg_matviews WHERE matviewname = 'v_infra_5179') THEN
            WITH m AS (
                SELECT s.sigungu_cd AS sgg_cd, s.geom
                FROM sgg_2023 s
            ), um AS (
                SELECT substring(s.sigungu_cd, 1, 4) AS sgg_cd_4, st_union(s.geom) AS geom
                FROM sgg_2023 s
                GROUP BY substring(s.sigungu_cd, 1, 4)
            ), r AS (
                SELECT sido_cd, sido_nm, sgg_cd, sgg_nm,
                       COUNT(fac_nm) AS fac_cnt,
                       geom,
                       COALESCE("year", '{year}') AS "year"
                FROM (
                    SELECT i.*, COALESCE(m.geom, um.geom) AS geom
                    FROM (
                        SELECT *
                        FROM infra
                        WHERE {infras.get(key)}
                    ) i
                    LEFT JOIN m ON i.sgg_cd = m.sgg_cd
                    LEFT JOIN um ON substring(i.sgg_cd, 1, 4) = um.sgg_cd_4
                ) sub
                WHERE geom IS NOT NULL AND sgg_nm != 'nan' AND sgg_nm IS NOT NULL
                GROUP BY sido_cd, sido_nm, sgg_cd, sgg_nm, "year", geom
            ), aggregated AS (
                SELECT sido_cd, sido_nm, sgg_cd, sgg_nm, SUM(fac_cnt) AS fac_cnt, geom
                FROM r
                WHERE "year"::INTEGER <= {year}
                GROUP BY sido_cd, sido_nm, sgg_cd, sgg_nm, geom
            )
            SELECT row_number() OVER () AS fid, sgg.*, COALESCE(a.fac_cnt, 0) AS fac_cnt, TO_TIMESTAMP('{year}', 'YYYY') AS base_year 
            FROM sgg_2023 sgg
            LEFT JOIN aggregated a ON sgg.sigungu_cd = a.sgg_cd;
        END IF;
    END $$;
    '''
    _util.execute_sql(sql)


for year in years:
    for key in infras.keys():
        drop_view(year, key)

for year in years:
    for key in infras.keys():
        # drop_view(year)
        infra_sql(year, key)


DependentObjectsStillExist: 오류:  기타 다른 개체들이 이 개체에 의존하고 있어, v_road_infra2000_5179 구체화된 뷰 삭제할 수 없음
DETAIL:  v_infra_5179 구체화된 뷰 의존대상: v_road_infra2000_5179 구체화된 뷰
HINT:  이 개체와 관계된 모든 개체들을 함께 삭제하려면 DROP ... CASCADE 명령을 사용하십시오
CONTEXT:  SQL 구문: "drop materialized view v_road_infra2000_5179"
PL/pgSQL 함수 "inline_code_block" 의 4번째 SQL 문


In [ ]:
import _util

def union_infra(year):

    sql = f'''
        create materialized view v_infra{year} as 
        with s as (
            select s.sigungu_cd as sgg_cd, s.sigungu_nm as sgg_nm, s.geom 
            from sgg_2023 s
        )
        select s.sgg_cd, sgg_nm
        , vci.fac_cnt as "통신구", vri.fac_cnt as "도로시설", vct.fac_cnt as "공동구", vgi.fac_cnt as "가스시설"
        , vri2.fac_cnt as "철도시설", vhi.fac_cnt as "열공급시설", s.geom
        from s
        left join v_cd_infra{year}_5179 vci on vci.sigungu_cd = s.sgg_cd
        left join v_road_infra{year}_5179 vri on vri.sigungu_cd = s.sgg_cd
        left join v_cmm_tunnel_infra{year}_5179 vct on vct.sigungu_cd = s.sgg_cd
        left join v_gas_infra{year}_5179 vgi on vgi.sigungu_cd = s.sgg_cd
        left join v_rail_infra{year}_5179 vri2 on vri2.sigungu_cd = s.sgg_cd
        left join v_heat_infra{year}_5179 vhi on vhi.sigungu_cd = s.sgg_cd
    '''

    _util.execute_sql(sql)


In [8]:
df = pd.read_excel(out_put_dir_path + '\\' + 'infra_road' + '.xlsx')

new_list = []
for idx, row in df.iterrows():
    if len(row['year']) < 8: new_list.append(row)

df = pd.DataFrame(new_list)
df.to_excel(out_put_dir_path + '\\' + 'infra_road_error' + '.xlsx')
